In [ ]:
%cd ~/code/projects/nn-play/

import keras 
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam, Adadelta, RMSprop, SGD
from keras.layers import Input, Dense, Convolution2D, Deconvolution2D, MaxPooling2D,\
    UpSampling2D, Merge, LSTM, Flatten, ZeroPadding2D, Reshape, BatchNormalization, Dropout
from keras.layers.wrappers import TimeDistributed

from structured_container import DataContainer

import tqdm
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
IM_WIDTH = 28
IM_HEIGHT = 28
IM_CHANNELS = 1

SERIES_SHIFT = 1
EP_LEN = 100 - SERIES_SHIFT
BATCH_SIZE = 32

V_SIZE = 128

In [ ]:
# predictive ae

input_im = Input(shape=(IM_WIDTH, IM_HEIGHT, IM_CHANNELS))
h = Convolution2D(16, 5, 5, subsample=(2,2), activation='relu', border_mode='same')(input_im)
h = Convolution2D(8, 3, 3, subsample=(2,2), activation='relu', border_mode='same')(h)
h = Flatten()(h)
v = Dense(V_SIZE, activation='relu')(h)

enc = Model(input_im, v)
enc.summary();

In [ ]:
input_v = Input(shape=(V_SIZE,))

h = Dense(8*7*7, activation='relu')(input_v)
h = Reshape((7,7,8))(h)
h = UpSampling2D((2,2))(h)
h = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(h)
h = UpSampling2D((2,2))(h)
output_im = Convolution2D(1, 5, 5, activation='sigmoid', border_mode='same')(h)

# output_im

dec = Model(input_v, output_im)
dec.summary();

In [ ]:
input_im = Input(shape=(IM_WIDTH, IM_HEIGHT, IM_CHANNELS))
v = enc(input_im)
output_im = dec(v)

ae = Model(input_im, output_im)
ae.summary();
ae.compile(optimizer=Adam(lr=0.0002), loss='mse')

In [ ]:
pred_ae = Sequential()
pred_ae.add(TimeDistributed(enc, input_shape=((EP_LEN-SERIES_SHIFT, IM_WIDTH, IM_HEIGHT, IM_CHANNELS))))
pred_ae.add(LSTM(V_SIZE, input_shape=(EP_LEN, V_SIZE), return_sequences=True))

# pred_ae.add(dec)
pred_ae.add(TimeDistributed(dec, input_shape=((EP_LEN, V_SIZE))))

# predae = Model(input_series, v_series)
pred_ae.summary();
pred_ae.compile(optimizer=Adam(lr=0.0002), loss='mse')

In [ ]:
images_train = DataContainer('data-balls/balls-train.pt', batch_size=32,
                          ep_len_read=EP_LEN)
images_valid = DataContainer('data-balls/balls-valid.pt', batch_size=32,
                          ep_len_read=EP_LEN)


In [ ]:
for p in np.linspace(0.1, 0.9, 10):
#     for j in tqdm.tqdm(range(200)):
    for j in range(500):
        x = images_train.get_n_random_episodes(32)
        x_noised = np.copy(x)
        for_removal = np.random.random(EP_LEN) < p
        for_removal[0:15] = False
        x_noised[:, for_removal] = 0
        pred_ae.train_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])

        
    x = images_valid.get_n_random_episodes(32)
    x_noised = np.copy(x)
    for_removal = np.random.random(EP_LEN) < p
    for_removal[0:15] = False
    x_noised[:, for_removal] = 0
    test_perf = pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
    print('p', p)
    print('test perf', test_perf)
    
    

In [ ]:
p = 0.7
for j in tqdm.tqdm(range(2000)):
    x = images_train.get_n_random_episodes(32)
    x_noised = np.copy(x)
    for_removal = np.random.random(EP_LEN) < p
    for_removal[0:15] = False
    x_noised[:, for_removal] = 0
    pred_ae.train_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])


x = images_valid.get_n_random_episodes(32)
x_noised = np.copy(x)
for_removal = np.random.random(EP_LEN) < p
for_removal[0:15] = False
x_noised[:, for_removal] = 0
test_perf = pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
print('p', p)
print('test perf', test_perf)
    

In [ ]:
x = images_valid.get_n_random_episodes(32)
x_noised = np.copy(x)
for_removal = np.random.random(EP_LEN) < p
for_removal[0:15] = False
x_noised[:, for_removal] = 0
test_perf = pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
print('p', p)
print('test perf', test_perf)

In [ ]:
# for i in tqdm.tqdm(range(2000)):
#    x = images_train.get_n_random_episodes(32)
#    pred_ae.train_on_batch(x[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])

In [ ]:
x = images_valid.get_n_random_episodes(32)
x_noised = np.copy(x)
for_removal = np.random.random(EP_LEN) < 0.99
for_removal[0:15] = False
x_noised[:, for_removal] = 0
pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
# pred_ae.test_on_batch(x[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])

In [ ]:
# y = pred_ae.predict(x[:,0:-SERIES_SHIFT,...])
y = pred_ae.predict(x_noised[:,0:-SERIES_SHIFT,...])
i = SERIES_SHIFT

In [ ]:
# gif generation
import imageio
images = []
for i in range(EP_LEN-SERIES_SHIFT):
    im_true = x[0,i+SERIES_SHIFT,:,:,0]
    im_noised = x_noised[0,i+SERIES_SHIFT,:,:,0]
    im_gen = y[0,i,:,:,0]
    im_diff = np.abs(im_true - im_gen)
    glued = np.concatenate([im_true,
                            im_gen,
                            im_noised,
                            im_diff],
                           axis=1)
# plt.imshow(glued, cmap='gray')
    images.append(glued)
imageio.mimsave('movie.gif', images)

In [ ]:
i=1

In [ ]:
i += 1
plt.imshow(x[0,i,:,:,0], cmap='gray', vmin=0.0, vmax=1.0)

In [ ]:
plt.imshow(np.abs(x[0,i,:,:,0]-y[0,i-SERIES_SHIFT,:,:,0]), cmap='gray', vmin=0.0, vmax=1.0)

In [ ]:
plt.imshow(y[0,i-SERIES_SHIFT,:,:,0], cmap='gray', vmin=0.0, vmax=1.0)


In [ ]:
fpath = 'models/pred_ae_single_init.hdf5'
pred_ae.save_weights(fpath)
# pred_ae.load_weights(fpath)

In [ ]:
images_train = DataContainer('data-balls/balls-train.pt', batch_size=32,
                          ep_len_read=EP_LEN)
images_valid = DataContainer('data-balls/balls-valid.pt', batch_size=32,
                          ep_len_read=EP_LEN)


In [ ]:
images_train.populate_images()

In [ ]:
x = images_train.get_batch_episodes()
i = 0

In [ ]:
plt.imshow(x[0,i,:,:,0], cmap='gray', vmin=0.0, vmax=1.0)
i += 1